In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-11 00:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-10 19:21:12,058 INFO: Initializing external client
2025-05-10 19:21:12,059 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:21:12,899 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.83s) 


In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,JC104,2.0,2025-05-11 00:00:00+00:00
1,HB608,1.0,2025-05-11 00:00:00+00:00
2,JC020,1.0,2025-05-11 00:00:00+00:00
3,HB302,2.0,2025-05-11 00:00:00+00:00
4,JC077,2.0,2025-05-11 00:00:00+00:00
...,...,...,...
78,JC081,1.0,2025-05-11 00:00:00+00:00
79,HB409,0.0,2025-05-11 00:00:00+00:00
80,JC074,1.0,2025-05-11 00:00:00+00:00
81,HB404,1.0,2025-05-11 00:00:00+00:00


In [8]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-10 19:22:06,909 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:22:06,911 INFO: Initializing external client
2025-05-10 19:22:06,911 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:22:07,554 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Current UTC time: 2025-05-10 23:22:06.909008+00:00
Next hour: 2025-05-11 00:00:00+00:00
Querying for date range: 2025-05-11 to 2025-05-12
Filtering for hour: 2025-05-11 00:00
Found 83 records


In [9]:
now = datetime.now(timezone.utc)

In [10]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,JC104,2.0,2025-05-11 00:00:00+00:00
1,HB608,1.0,2025-05-11 00:00:00+00:00
2,JC020,1.0,2025-05-11 00:00:00+00:00
3,HB302,2.0,2025-05-11 00:00:00+00:00
4,JC077,2.0,2025-05-11 00:00:00+00:00
...,...,...,...
78,JC081,1.0,2025-05-11 00:00:00+00:00
79,HB409,0.0,2025-05-11 00:00:00+00:00
80,JC074,1.0,2025-05-11 00:00:00+00:00
81,HB404,1.0,2025-05-11 00:00:00+00:00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  83 non-null     object                 
 1   predicted_demand    83 non-null     float64                
 2   pickup_hour         83 non-null     datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), object(1)
memory usage: 2.1+ KB


In [12]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [13]:
dt = current_date.ceil('h')


In [16]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-05-10 05:00:00")  
results = query.read()

2025-05-10 19:28:25,442 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:28:25,444 INFO: Initializing external client
2025-05-10 19:28:25,444 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:28:26,155 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 


In [19]:
fg.filter((fg.pickup_hour >= "2023-02-13") & (fg.pickup_hour < "2025-05-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.79s) 


,pickup_location_id,predicted_demand,pickup_hour
0,JC104,2.0,2025-05-11 00:00:00+00:00
1,HB608,1.0,2025-05-11 00:00:00+00:00
2,JC020,1.0,2025-05-11 00:00:00+00:00
3,HB302,2.0,2025-05-11 00:00:00+00:00
4,JC077,2.0,2025-05-11 00:00:00+00:00
...,...,...,...
78,JC081,1.0,2025-05-11 00:00:00+00:00
79,HB409,0.0,2025-05-11 00:00:00+00:00
80,JC074,1.0,2025-05-11 00:00:00+00:00
81,HB404,1.0,2025-05-11 00:00:00+00:00


In [18]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,JC104,2.0,2025-05-11 00:00:00+00:00
1,HB608,1.0,2025-05-11 00:00:00+00:00
2,JC020,1.0,2025-05-11 00:00:00+00:00
3,HB302,2.0,2025-05-11 00:00:00+00:00
4,JC077,2.0,2025-05-11 00:00:00+00:00
...,...,...,...
78,JC081,1.0,2025-05-11 00:00:00+00:00
79,HB409,0.0,2025-05-11 00:00:00+00:00
80,JC074,1.0,2025-05-11 00:00:00+00:00
81,HB404,1.0,2025-05-11 00:00:00+00:00


In [20]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [21]:
predictions = fetch_next_hour_predictions()

2025-05-10 19:31:11,186 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:31:11,189 INFO: Initializing external client
2025-05-10 19:31:11,189 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:31:11,832 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Current UTC time: 2025-05-10 23:31:11.186497+00:00
Next hour: 2025-05-11 00:00:00+00:00
Found 83 records


In [22]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-05-10 19:31:23,166 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:31:23,169 INFO: Initializing external client
2025-05-10 19:31:23,169 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:31:23,866 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000


In [23]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [24]:
df["pickup_hour"].max()

Timestamp('2025-05-11 00:00:00+0000', tz='Etc/UTC')

In [25]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-11 00:00:00+00:00


In [26]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
0,JC104,2.0,2025-05-11 00:00:00+00:00
1,HB608,1.0,2025-05-11 00:00:00+00:00
2,JC020,1.0,2025-05-11 00:00:00+00:00
3,HB302,2.0,2025-05-11 00:00:00+00:00
4,JC077,2.0,2025-05-11 00:00:00+00:00
...,...,...,...
78,JC081,1.0,2025-05-11 00:00:00+00:00
79,HB409,0.0,2025-05-11 00:00:00+00:00
80,JC074,1.0,2025-05-11 00:00:00+00:00
81,HB404,1.0,2025-05-11 00:00:00+00:00
